In [1]:
from dotenv import load_dotenv
from pprint import pprint
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

In [2]:
load_dotenv()

True

In [3]:
model = ChatOpenAI(
    model_name="gpt-4o"
)

In [4]:
prompt_sentiment = ChatPromptTemplate.from_messages(
    [
        (
            "system" , "You are a helpful assistant",
        ),
        (
            "human", "Classify the sentiment of this comment as positive, negative, or neutral: {review}. you must answer either positive, negative, neutral, nothing else."
        )
    ]
)

In [5]:
task_lowercase = RunnableLambda(lambda text: text.lower())

In [6]:
task_sentiment = (
    prompt_sentiment
    | model
    | StrOutputParser()
    | task_lowercase
)

In [7]:
sentiment = task_sentiment.invoke(
    {
        "review" : "This product is awesome"
    }
)

In [8]:
print(sentiment)

positive


In [9]:
sentiment = task_sentiment.invoke(
    {
        "review" : "This product is really bad"
    }
)

In [10]:
print(sentiment)

negative


In [11]:
sentiment = task_sentiment.invoke(
    {
        "review" : "This product is okay"
    }
)

In [12]:
print(sentiment)

neutral


In [13]:
system_msg = "you are a customer representative who reads customer's review and respond to their product review"

In [14]:
prompt_positive = ChatPromptTemplate.from_messages(
    [
        (
            "system", system_msg,
        ),
        (
            "human","Generate a thank-you note for this positive review: {review}."
        )
    ]
)

In [16]:
prompt_negative = ChatPromptTemplate.from_messages(
    [
        (
            "system", system_msg,
        ),
        (
            "human","Generate a response addressing this negative review: {review}. Ask if the customer wants a refund, a replacement, or something else."
        )
    ]
)

In [17]:
prompt_neutral = ChatPromptTemplate.from_messages(
    [
        (
            "system", system_msg,
        ),
        (
            "human","Generate a response for more details for this neutral review: {review}. Ask what needs to be done to make the product."
        )
    ]
)

In [18]:
prompt_default = ChatPromptTemplate.from_messages(
    [
        (
            "system", system_msg,
        ),
        (
            "human","Don't do anything for this review: {review}"
        )
    ]
)

In [19]:
from langchain_core.runnables import RunnableBranch

In [20]:
task_branch = RunnableBranch(
    (
        lambda review: review == "positive", prompt_positive | model | StrOutputParser()
    ),
    (
        lambda review: review == "negative", prompt_negative | model | StrOutputParser()
    ),
    (
        lambda review: review == "neutral", prompt_neutral | model | StrOutputParser()
    ),
    prompt_default | model | StrOutputParser()
)

In [21]:
chain = (prompt_sentiment | model | StrOutputParser() | task_branch)

In [22]:
out = chain.invoke(
    {
        "review": "I love this product!"
    }
)

In [23]:
print(out)

**Customer Review:**

"I'm really disappointed with the quality of this product. It stopped working properly after just a few uses, which is not what I expected at all. I was really hoping for better durability, given the price."

---

**Response:**

Dear [Customer Name],

Thank you for taking the time to share your experience with us. We sincerely apologize for any inconvenience caused by the issues you've encountered with your purchase. At [Company Name], we strive to ensure that our products meet the highest standards of quality and longevity, and we regret that this was not your experience.

We are committed to making things right for you. Please let us know if you would prefer a refund, a replacement, or if there is something else we can do to address your concerns. Your satisfaction is our top priority, and we want to ensure you have a positive experience with our company.

Thank you once again for your feedback, and we hope to resolve this matter to your satisfaction soon.

Warm

In [24]:
out = chain.invoke(
    {
        "review": "This product is terrible. It no longer functions just after one day of use"
    }
)

In [25]:
print(out)

Certainly! Here's a potential response you can use:

---

Dear [Customer's Name],

Thank you for taking the time to share your feedback with us. We are genuinely sorry to hear that your experience with our product did not meet your expectations. Your satisfaction and experience are very important to us, and we would like to make things right.

Could you please let us know if you would prefer a refund, a replacement, or if there is something else we can do to resolve this issue for you? We are here to assist you and ensure your concerns are addressed promptly.

Please reach out to us at your earliest convenience so we can work towards a resolution that meets your needs.

Thank you for giving us the opportunity to make this right.

Best regards,

[Your Name]  
[Your Position]  
[Company Name]  
[Contact Information]  

--- 

Feel free to adjust the details to fit your specific context.


In [28]:
out = chain.invoke(
    {
        "review": "This product is so so. neither grate or bad, it does what it's supposed to do."
    }
)

In [29]:
print(out)

Thank you for your feedback! If you have any thoughts or specific experiences you'd like to share about the product, feel free to let us know. We appreciate your input and are here to assist if you have any questions or need further assistance.


RunnablePassthrough

In [30]:
model = ChatOpenAI(
    model_name = "gpt-4o-mini",
    temperature=0
)

In [33]:
from langchain.prompts import PromptTemplate

In [34]:
prompt = PromptTemplate.from_template("{number}에 5를 더한 결과값을 알려줘")

In [35]:
chain = (
    prompt
    |model
    |StrOutputParser()
)

In [36]:
out = chain.invoke(
    {"number":9}
)

In [37]:
out

'9에 5를 더하면 14입니다.'

In [38]:
from langchain_core.runnables import RunnablePassthrough

In [39]:
chain = (
    {"number": RunnablePassthrough() }
    |prompt
    |model
    |StrOutputParser()
)

In [40]:
out = chain.invoke(9)

In [41]:
out

'9에 5를 더하면 14입니다.'